# 🔅Реализуем алгоритм KNN - самостоятельно

## Алгоритм
* Посчитать расстояние от каждой из 38 точек до 112 точек, по которым знаем тип цветка.
* Расстояние рассчитывается по всем признакам ($\sqrt{(x_1 - x_2)^2...}$) - где индексы это тренировочные и тестовые данные, а $x$ - это фичи (первая фича x, вторая y, и т.д.).
* Список с расстояниями - состоит из расстояния (положительное число) до цветка и типа цветка от которого считали расстояние.
* Сделать сортировку в данном списке (нужна `lambda` т.к. сложная структура вложенные списки).
* Взять $k$ значений из сортированного списка.
* Посчитать сколько и каких типов среди выбранных значений, выбрать тот тип, что чаще других встречается.
* Сделать предсказание и далеко идёщие выводы.
* Метрика проверки - угадал к общему количеству классифицируемых объектов.

In [1]:
import numpy as np
# Получаем данные о цветах
from sklearn.datasets import load_iris
iris_dataset = load_iris()
iris_dataset['feature_names']
# Попробуем повторить результат преподавателя по фичам petal length, petal width - 2 и 3
print(f"Форма представленных данных {iris_dataset['data'].shape}")
# Сделаем срез по многомерному массиву последние два столбца
petal = iris_dataset['data'][:, 2:]
target = iris_dataset['target'].reshape(150, 1)
# Не забывать про скобки вокургу np.arrays
data = np.concatenate([target, petal], axis=1)
print(f"Получаем следующую структуру (y, x_1, x_2): {data[:2]}")

Форма представленных данных (150, 4)
Получаем следующую структуру (y, x_1, x_2): [[0.  1.4 0.2]
 [0.  1.4 0.2]]


In [2]:
# Проведём разделение данных на тренировочные и тестовые данные
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state=0)
print(f'Обучающие данные {train.shape}, Тестовые (в которых будет предсказывать) данные {test.shape}')

Обучающие данные (112, 3), Тестовые (в которых будет предсказывать) данные (38, 3)


In [3]:
# используем класс, который позволяет подсчитывать колчество повторяющихся значений в списке
from collections import Counter
# Определим функцию предсказывающую типы цветков для данных передаваемых через test,
# на оснвое известных типов в train
def k_predict(train=train, test=test, k=5):
    # Итоговый результат список состоящий из типов цветоков (38 элементов)
    result_items = []
    # Для всех данных (38), которые нужно проклассифицировать
    for item in test:
        # Считаем расстояния и записываем классы ццветков [[1, 10],[0, 30], ...]
        distance = []
        for known_type in train:
            # данные обрабатываемые имеют структуру np.array и на нулевом месте записан тип цветка
            # Расчёт расстояния квадртаный корень заменим на возведение в степень 1\2
            dist = (np.sum((item[1:] - known_type[1:])**2))**0.5
            type_and_dist = [int(known_type[0]), dist]
            distance.append(type_and_dist)
        # Сортировка сложного объекта по второму значению
        distance.sort(key=lambda i: i[1])
        # Только тип цветка из посчитанных расстояний k элементов в начале
        result_types = [i[0] for i in distance][:k]
        # Наиболее часто встречающийся в списке тип (без количества)
        result_item = Counter(result_types).most_common(n=1)[0][0]
        # Результат предсказания для n-ого цветка
        result_items.append(result_item)
    # решейп для того, чтобы получить ту же форму, 
    # что и данные (входные) для которых делаются эти предсказания    
    return np.array(result_items).reshape(38,1)

In [4]:
prediction = k_predict(train=train, test=test, k=5)

In [5]:
def accuracy(test=test, prediction=prediction):
    # Сравниваем предсказанные значения с реальными
    pred = np.sum(test[:, 0:1] == prediction)
    acc = pred/(len(test))
    print(f'Точность предсказаний: {acc}')

accuracy(test=test, prediction=prediction)

Точность предсказаний: 0.9736842105263158


# 🔅🔅 Реализуем алгоритм «Наивного Байеса» - самостоятельно

## Краткое описание алгоритма
1. Нужно посчитать данные по 3 классам цветов по фичам (в нашем случае 2 фичи).
2. Нужно рассчитать математическое ожидание - средняя $\mu$ - для каждого класса.
3. Нужно рассчитать дисперсию - вариацию $\sigma$ - для каждого класса.
  * Данные параметры задают форму нормального распределения (полностью её определяют).
4. Реализовать формулу (рассчёт веротяности) $p(x_i|y) = \frac{1}{\sqrt{2\pi \sigma^2_y}}exp(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y})$
  * Нужно зафиксировать параметры $\sigma$ и $\mu$ - по данным из обучающей выборки.
  * Значения фичей из обучающей выборки больше не понадабятся.
5. Посчитать условную вероятность по формуле Байеса $P(Class\ A|Feature\ 1, Feature\ 2) = P(Feature\ 1|Class\ A)\cdot P(Feature\ 2|Class\ A)\cdot P(Class\ A)$
  * Умножение заменим на сложение логарифмов, чтобы не потерять в точности.
  * Не забыть также рассчитать (долю цветков определённого класса - в разбивке - вероятность отнесения к классу) - $P(Class\ A)$
6. Повторить расчёт для каждого цветка по трём классам, из получившихся вероятностей выбрать наибольшую.  
  
[P.S. Реализация алгорита на классах и немного теории](https://www.youtube.com/watch?v=BqUmKsfSWho)

In [6]:
import numpy as np
# Получаем данные о цветах
from sklearn.datasets import load_iris
iris_dataset = load_iris()
iris_dataset['feature_names']
# Попробуем повторить результат преподавателя по фичам petal length, petal width - 2 и 3
print(f"Форма представленных данных {iris_dataset['data'].shape}")
# Сделаем срез по многомерному массиву последние два столбца
petal = iris_dataset['data'][:, 2:]
target = iris_dataset['target'].reshape(150, 1)

# Проведём разделение данных на тренировочные и тестовые данные
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(petal, target, random_state=0)

# train, test = train_test_split(data, random_state=0)
print(f'Обучающие данные {train_x.shape}, Тестовые (в которых будет предсказывать) данные {test_x.shape}')
print(f'Пример структуры тренировчных данных:{train_x[0]}, {train_y[0]}')
print(f'Пример структуры тестовых данных:{test_x[0]}, {test_y[0]}')

Форма представленных данных (150, 4)
Обучающие данные (112, 2), Тестовые (в которых будет предсказывать) данные (38, 2)
Пример структуры тренировчных данных:[4.2 1.5], [1]
Пример структуры тестовых данных:[5.1 2.4], [2]


In [7]:
# np.where() - делает отбор по условию, возвращает индексы элементов.
# Разделим тренировочные данные на 3 класса
# Обязательно нужен [0], возвращаеся 2D массив при отборе
class_0 = train_x[np.where(train_y == 0)[0]]
class_1 = train_x[np.where(train_y == 1)[0]]
class_2 = train_x[np.where(train_y == 2)[0]]
print(f'Проводим разделение на 3 класса тренировочных данных')
print(f'Класс 0: {class_0[0]}, {class_0.shape}')
print(f'Класс 1: {class_2[0]}, {class_1.shape}')
print(f'Класс 2: {class_2[0]}, {class_2.shape}')

Проводим разделение на 3 класса тренировочных данных
Класс 0: [1.3 0.2], (37, 2)
Класс 1: [5.5 2.1], (34, 2)
Класс 2: [5.5 2.1], (41, 2)


In [8]:
# Проведём расчёт мат.ожидания, дисперсии и вероятности для трёх классов цветов
def eva_param(data):
    # Получаем мат.ожидание по двум фичам для каждого класса
    mu = data.mean(axis=0)
    # Получаем дисперсию (вариацию) по двум фичам для каждого класса
    sigma    = data.var(axis=0)
    # Частота - вероятность отнесения к классу
    freq = data.shape[0]/112.0
    # Соберём параметры в список по фичам
    params = [(s, m, freq) for s, m in zip(sigma, mu)]
    return params

class_0_param = eva_param(class_0)
class_1_param = eva_param(class_1)
class_2_param = eva_param(class_2)
print(f'Получаем параметры для каждого класса. (sigma - var, mu - mean, prob_class): \n {class_0_param}')

Получаем параметры для каждого класса. (sigma - var, mu - mean, prob_class): 
 [(0.019780861943024117, 1.454054054054054, 0.33035714285714285), (0.011599707815924029, 0.24054054054054055, 0.33035714285714285)]


## Формула для расчёта вероятности - плотность нормального распределения
$$p(x_i|y) = \frac{1}{\sqrt{2\pi \sigma^2_y}}exp^{(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y})}$$  
* $e$ в степени

In [9]:
import math
def probability_counter(x, sigma, mu):
    part_1 = (1 / (math.sqrt(2*math.pi*sigma)))
    part_2 = math.e**(-((x - mu)**2)/(2*sigma))
    px_i = part_1 * part_2
    return px_i

    # Фрмула из https://www.youtube.com/watch?v=BqUmKsfSWho
    # Результат тот же, но в расчётах вероятностей есть расхождение, видимо из-за точности
#     numerator = np.exp(-((x-mu)**2) / (2 * sigma))
#     denominator = np.sqrt(2 * np.pi * sigma)
#     return float(numerator * denominator)

In [10]:
def fit(params, test):
    f_1, f_2 = params
    pp = []
    
    sigma1, mu1, p_y = f_1
    sigma2, mu2, p_y = f_2
    
    for xf1, xf2 in zip(test_x[:, 0], test_x[:, 1]):
        p1 = probability_counter(xf1, sigma=sigma1, mu=mu1)
        p2 = probability_counter(xf2, sigma=sigma2, mu=mu2)
        # Не влияет на метрику точности, тот же результат 0,97, но веротяность будет не в долях.
        probability = np.log(p1) + np.log(p2) + np.log(p_y)
        # Прямой расчёт - риск потери точности
#         probability = p1*p2*p_y
        pp.append(probability) 
    return pp

# Получаем данные по всем тестовым данным по каждому классу
cl0 = fit(params=class_0_param, test=test_x)
cl1 = fit(params=class_1_param, test=test_x)
cl2 = fit(params=class_2_param, test=test_x)
# Всё объединить
cl_result = [[i, j, k] for i, j, k in zip(cl0, cl1, cl2)]

In [11]:
# Определеить какой класс - самое большое занчение вероятности
predict = []
for item in cl_result:
    volume = max(item)
    flower_id = item.index(volume)
    predict.append(flower_id)
    
print(f'Пример предсказания:')
print(f'{item} - {flower_id} - {volume}')
print(f'Сделаем обратный переход от логарифмов к долям:')
print(f'{list(map(math.exp, item))} - Наибольшая вероятность принадлежности к классу: {flower_id}')
print('=======================================')
score = 0
for i, j in zip(predict, list(test_y[:, 0])):
    if i == j:
        score += 1
print(f'Точность предсказания {score/len(predict)}')
print('**************************************')
# Сделаем проверку по библиотечной реализации
from sklearn.naive_bayes import GaussianNB
flower = GaussianNB()
flower_model_1 = flower.fit(train_x, train_y.ravel())
nb_predictions_1 = flower.predict(test_x)
accuracy_1 = flower.score(test_x, test_y.ravel())
print(f'Точность библиотечной реализации: {accuracy_1}')

Пример предсказания:
[-414.42294947975057, -3.580900585964839, -2.735467638613015] - 2 - -2.735467638613015
Сделаем обратный переход от логарифмов к долям:
[1.0432775616604588e-180, 0.02785060509360884, 0.06486366725498957] - Наибольшая вероятность принадлежности к классу: 2
Точность предсказания 0.9736842105263158
**************************************
Точность библиотечной реализации: 0.9736842105263158


# 🔅🔅🔅 Алгоритм «Дерево решений»

## Краткое описание алгоритма  
  
1. Посчитать меру энтропии - $H(X) = - \sum_i P_x(x_i)log_b P_x(x_i)$
  * основание логарифма $2$ или $e$.
  * P - вероятность, можно использовать частотный подход.
  * $P_x(x_i)$ - вероятность класса (цветков класса к общему количеству)
  * Если Энтропия равна 0, или не очень большая остановить алгоритм.
  * Алгоритм рекурсивный, также для остановки можно использовать кол-во ветвей.
2. Выбираем критерий (фичу) у объекта, и по этому критерию разделяем датасет на две части.
  * *Например: Длина стебля - больше, чем 1.0?*
  * Считаем для всех объектов в наборе данных, т.е. разеделяем очень много раз на две части.
3. Считаем полученную энтропию при каждом разделении.
4. Выбираем разделение, которое больше всего уменьшает энтропию.
5. Данное разделение - формирует правило, которое нужно сохранить.
6. Всё повторяем до точки остановки.
7. Нужно сохранить полученные правила и использовать их для классификации тестовых данных.
8. Правила можно хранить в объектах, которые будут содержать в себе правило по частям и ссылку на следующий объект-правило.
  
[P.S. Видео объяснение](https://www.youtube.com/watch?v=RmajweUFKvM)

In [12]:
# todo написать на досуге